## before submission
- remove all TEST units
- resolve all TODO's

In [1]:
import sys
from collections import defaultdict
from collections import Counter
import math
import random
import os
import os.path
"""
COMS W4705 - Natural Language Processing - Fall 2023 
Programming Homework 1 - Trigram Language Models
Daniel Bauer
"""

'\nCOMS W4705 - Natural Language Processing - Fall 2023 \nProgramming Homework 1 - Trigram Language Models\nDaniel Bauer\n'

In [2]:
# lazy evaluation, not storing but printing on the fly when needed
# only prints once per generator
# lexicon: python set; all tokens not in this set are replaced with "UNK"

def corpus_reader(corpusfile, lexicon=None):
    with open(corpusfile,'r') as corpus: 
        for line in corpus: 
            if line.strip():
                sequence = line.lower().strip().split()
                if lexicon: 
                    yield [word if word in lexicon else "UNK" for word in sequence]
                else: 
                    yield sequence

In [3]:
# # TEST corpus reader

# test_cr = corpus_reader("./hw1_data/brown_test.txt")
# for s in test_cr:
#     print(type(s))
#     print(s)
#     break

<class 'list'>
['the', 'blade', 'came', 'out', 'with', 'a', 'snick', '!']


In [4]:
# extract a set of lexicon from a file

def get_lexicon(corpus):
    word_counts = defaultdict(int)
    for sentence in corpus:
        for word in sentence: 
            word_counts[word] += 1
    return set(word for word in word_counts if word_counts[word] > 1)

In [5]:
def get_ngrams(sequence, n):
    """
    COMPLETE THIS FUNCTION (PART 1)
    Given a sequence, this function should return a list of n-grams, where each n-gram is a Python tuple.
    This should work for arbitrary values of n >= 1 
    """
    s = list(sequence) # must make a deep copy
    # add START and STOP token to sequence
    if (n == 1):
        s.insert(0, "START")
    else:
        for i in range(n-1):
            s.insert(0,"START")
    s.append("STOP")

    # produce ngrams
    ngrams = []
    for i in range(len(s)-n+1):
        ngram = []
        for j in range(n):
            ngram.append(s[i+j])
        ngrams.append(tuple(ngram))
    return ngrams

In [6]:
# # TEST part1 get_ngrams

# s = ['the', 'blade', 'came', 'out', 'with', 'a', 'snick', '!']
# # s = ['natural', 'language', 'processing']
# # get_ngrams(s, 1)
# # get_ngrams(s, 2)
# get_ngrams(s, 3)

[('START', 'START', 'the'),
 ('START', 'the', 'blade'),
 ('the', 'blade', 'came'),
 ('blade', 'came', 'out'),
 ('came', 'out', 'with'),
 ('out', 'with', 'a'),
 ('with', 'a', 'snick'),
 ('a', 'snick', '!'),
 ('snick', '!', 'STOP')]

In [62]:
class TrigramModel(object):
    total_word_count = 0
    total_sentence_count = 0
    
    def __init__(self, corpusfile):
    
        # Iterate through the corpus once to build a lexicon 
        generator = corpus_reader(corpusfile)
        self.lexicon = get_lexicon(generator)
        self.lexicon.add("UNK")
        self.lexicon.add("START")
        self.lexicon.add("STOP")

        # Now iterate through the corpus again and count ngrams
        generator = corpus_reader(corpusfile, self.lexicon)
        self.count_ngrams(generator)

    def count_ngrams(self, corpus):
        """
        COMPLETE THIS METHOD (PART 2)
        Given a corpus iterator, populate dictionaries of unigram, bigram,
        and trigram counts. 
        """   
        self.unigramcounts = Counter() # might want to use defaultdict or Counter instead
        self.bigramcounts = Counter()
        self.trigramcounts = Counter()

        for s in corpus:
            unigrams = get_ngrams(s, 1)
            bigrams = get_ngrams(s, 2)
            trigrams = get_ngrams(s, 3)
            for u in unigrams:
                # if(u[0] == "START"):
                #     continue
                self.unigramcounts[u] += 1
            for b in bigrams:
                self.bigramcounts[b] += 1
            for t in trigrams:
                self.trigramcounts[t] += 1
            self.total_sentence_count += 1
            self.total_word_count += (len(s)+1) # TODO: should I count "STOP" in total_word_count?
        
        return

    def raw_bigram_probability(self, bigram):
        """
        COMPLETE THIS METHOD (PART 3)
        Returns the raw (unsmoothed) bigram probability
        """
        bigram_den_count = self.unigramcounts[(bigram[0],)]
        if (bigram_den_count == 0):
            return 1/len(self.lexicon)
        bigram_num_count = self.bigramcounts[bigram]
        return bigram_num_count/bigram_den_count

    def raw_trigram_probability(self,trigram):
        """
        COMPLETE THIS METHOD (PART 3)
        Returns the raw (unsmoothed) trigram probability
        """
        trigram_den_count = self.bigramcounts[trigram[0:2]]
        if(trigram[0] == "START" and trigram[1] == "START"):
            return self.raw_bigram_probability(trigram[1:2])
        if(trigram_den_count == 0):
            return 1/len(self.lexicon)
        trigram_num_count = self.trigramcounts[trigram]
        return trigram_num_count/trigram_den_count
    
    def raw_unigram_probability(self, unigram):
        """
        COMPLETE THIS METHOD (PART 3)
        Returns the raw (unsmoothed) unigram probability.
        """
        return self.unigramcounts[unigram]/self.total_word_count

    def generate_sentence(self,t=20): 
        """
        COMPLETE THIS METHOD (OPTIONAL)
        Generate a random sentence from the trigram model. t specifies the
        max length, but the sentence may be shorter if STOP is reached.
        """
        return result            

    def smoothed_trigram_probability(self, trigram):
        """
        COMPLETE THIS METHOD (PART 4)
        Returns the smoothed trigram probability (using linear interpolation). 
        """
        lambda1 = 1/3.0
        lambda2 = 1/3.0
        lambda3 = 1/3.0

        # print(trigram, trigram[0:3:2], (trigram[0],))
        
        term1 = self.raw_trigram_probability(trigram)*lambda1
        term2 = self.raw_bigram_probability(trigram[1:3])*lambda2
        term3 = self.raw_unigram_probability((trigram[2],))*lambda3

        # print(term1, term2, term3)
        
        return term1 + term2 + term3
        
    def sentence_logprob(self, sentence):
        """
        COMPLETE THIS METHOD (PART 5)
        Returns the log probability of an entire sequence.
        """
        logprob = 0.0
        trigrams = get_ngrams(sentence, 3)
        for t in trigrams:
            prob = self.smoothed_trigram_probability(t)
            logprob += math.log2(prob)
        
        return logprob

    def perplexity(self, corpus):
        """
        COMPLETE THIS METHOD (PART 6) 
        Returns the log probability of an entire sequence.
        """
        s_logprob = 0.0
        lexicon_count = 0
        for s in corpus:
            s_logprob += self.sentence_logprob(s)
            lexicon_count += (len(s)+1)
        l =  s_logprob/lexicon_count
        
        return 2**(l*(-1))

In [54]:
# # TEST part 6 perplexity on test data
# model = TrigramModel("./hw1_data/brown_train.txt")
# generator = corpus_reader("./hw1_data/brown_test.txt", model.lexicon)
# print(model.perplexity(generator))

223.09550775726743


In [34]:
# # TEST part2 count_ngrams
# model = TrigramModel("./hw1_data/brown_train.txt")

# print(model.trigramcounts[('START','START','the')])
# print(model.bigramcounts[('START','the')])
# print(model.unigramcounts[('the',)])

5478
5478
61428


In [55]:
# # TEST part 6 perplexity on training data
# model = TrigramModel("./hw1_data/brown_train.txt")
# generator = corpus_reader("./hw1_data/brown_train.txt", model.lexicon)
# print(model.perplexity(generator))

16.438357509784264


In [20]:
# # TEST check raw probabilities
# model = TrigramModel("./hw1_data/brown_train.txt")
# total_unigram_prob = 0.0

# for i in model.unigramcounts:
#     total_unigram_prob += model.raw_unigram_probability(i)

# total_unigram_prob

In [56]:
# # TEST part3 probabailities
# model = TrigramModel("./hw1_data/brown_train.txt")

# print(model.raw_trigram_probability(('START','START','the')))
# print(model.raw_bigram_probability(('START','the')))
# print(model.raw_unigram_probability(('the',)))

0.0
0.13163839092613064
0.058920067333931216


In [57]:
# # TEST part4 smoothed_trigram_probability

# print(model.smoothed_trigram_probability(('START','START','the')))

0.06351948608668728


In [59]:
# # TEST part5 sentence log probability
# model = TrigramModel("./hw1_data/brown_train.txt")

# # print(model.s)
# # model.sentence_logprob(model.s)

In [65]:
def essay_scoring_experiment(training_file1, training_file2, testdir1, testdir2):

        model1 = TrigramModel(training_file1) # high
        model2 = TrigramModel(training_file2) # low

        total = 0
        correct = 0
 
        for f in os.listdir(testdir1): # high
            pp1 = model1.perplexity(corpus_reader(os.path.join(testdir1, f), model1.lexicon))
            pp2 = model2.perplexity(corpus_reader(os.path.join(testdir1, f), model2.lexicon))
            if (pp1 < pp2):
                correct += 1
            total += 1
            
    
        for f in os.listdir(testdir2): # low
            pp2 = model2.perplexity(corpus_reader(os.path.join(testdir2, f), model2.lexicon))
            pp1 = model1.perplexity(corpus_reader(os.path.join(testdir2, f), model1.lexicon))
            if (pp1 > pp2):
                correct += 1
            total += 1
        
        return correct/total

In [66]:
# # TEST part7 essay scoring

# train_high = "./hw1_data/ets_toefl_data/train_high.txt"
# train_low = "./hw1_data/ets_toefl_data/train_low.txt"
# dir_high = "./hw1_data/ets_toefl_data/test_high"
# dir_low = "./hw1_data/ets_toefl_data/test_low"

# essay_scoring_experiment(train_high, train_low, dir_high, dir_low)

0.8466135458167331

In [68]:
# if __name__ == "__main__":

#     model = TrigramModel(sys.argv[1]) 

#     # put test code here...
#     # or run the script from the command line with 
#     # $ python -i trigram_model.py [corpus_file]
#     # >>> 
#     #
#     # you can then call methods on the model instance in the interactive 
#     # Python prompt. 

    
#     # Testing perplexity: 
#     dev_corpus = corpus_reader(sys.argv[2], model.lexicon)
#     pp = model.perplexity(dev_corpus)
#     print(pp)


#     # Essay scoring experiment: 
#     acc = essay_scoring_experiment('train_high.txt', "train_low.txt", "test_high", "test_low")
#     print(acc)

FileNotFoundError: [Errno 2] No such file or directory: '-f'